In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ( confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score)

from matplotlib import pyplot as plt
from tqdm import tqdm

import torch
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import transformers

In [ ]:
# if torch.cuda.is_available():
#   dev = torch.device('cuda:0')
# else:
#   dev = torch.device('cpu')
dev = torch.device('cpu')

## Importing the dataset

In [ ]:
df = pd.read_csv('data.tsv', delimiter='\t', header=None)

In [ ]:
batch_1 = df[:2000]

In [ ]:
batch_1

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
1995,too bland and fustily tasteful to be truly pru...,0
1996,it does n't work as either,0
1997,this one aims for the toilet and scores a dire...,0
1998,in the name of an allegedly inspiring and easi...,0


In [ ]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

## Loading Pre-trained BERT model

In [ ]:
# DistilBERT:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

# BERT:
#model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
# print(tokenized)
for i in tokenized[:15].values:
    print(len(i))

20
16
45
22
25
21
21
17
28
7
22
24
29
16
18


In [ ]:
# calc maximum length of tokenized sentence

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

print(max_len)

59


In [ ]:
# Add Padding

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
padded

array([[  101,  1037, 18385, ...,     0,     0,     0],
       [  101,  4593,  2128, ...,     0,     0,     0],
       [  101,  2027,  3653, ...,     0,     0,     0],
       ...,
       [  101,  2023,  2028, ...,     0,     0,     0],
       [  101,  1999,  1996, ...,     0,     0,     0],
       [  101,  1996,  3185, ...,     0,     0,     0]])

In [ ]:
np.array(padded).shape

(2000, 59)

In [ ]:
# Masking: to tell BERT to ignore/mask added padding 

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

In [ ]:
## Tokentzed Sentence => BERT => Sentence Embedding

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask)

In [ ]:
# [CLS] Sentence [SEP]
# output corresponding to [CLS] token is embedding for entire sentence.
# saving features corresponding to [CLS]

features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
features

array([[-0.21593437, -0.14028914,  0.00831094, ..., -0.13694856,
         0.5867004 ,  0.20112717],
       [-0.17262724, -0.14476171,  0.00223437, ..., -0.17442554,
         0.21386446,  0.37197468],
       [-0.05063341,  0.07203971, -0.02959722, ..., -0.07148952,
         0.7185241 ,  0.26225477],
       ...,
       [-0.27829772, -0.2480361 ,  0.13585785, ..., -0.19039167,
         0.13099585,  0.3497837 ],
       [-0.03667716,  0.10638539, -0.0111102 , ..., -0.11206637,
         0.4161945 ,  0.50338024],
       [ 0.12402633,  0.01425167,  0.01038398, ..., -0.11606557,
         0.5345913 ,  0.27495337]], dtype=float32)

In [ ]:
features.shape

(2000, 768)

In [ ]:
labels = batch_1[1]

## Train Model


In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
pd.DataFrame(train_features)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.107823,0.111220,0.007751,-0.186264,0.017974,0.030283,0.243291,0.338382,-0.062372,-0.382746,-0.058209,0.004015,0.079638,0.205572,-0.141916,0.373760,0.087848,0.146198,0.149237,-0.111292,-0.091705,-0.160435,0.071609,0.062883,0.062110,0.128250,0.008427,-0.196469,0.027979,-0.016148,0.124949,0.289490,-0.097533,-0.147253,-0.096288,-0.084171,0.073260,0.030870,0.191073,0.083149,...,0.022878,0.104922,-0.235396,0.387556,-0.072939,-0.340098,-0.065424,-0.024178,-0.022620,-0.217506,0.183706,0.011744,0.369475,0.005822,0.223637,0.165307,0.156381,-0.197007,-0.013248,-0.065221,-0.089332,-0.189002,-0.005612,0.145734,-7.047712,-0.410940,-0.064804,-0.173091,-0.163102,-0.013241,0.030233,-0.039298,0.070220,0.008022,0.264552,0.034919,-0.051451,0.101448,0.284157,0.335191
1,0.095286,0.077067,-0.264647,-0.044359,-0.067140,-0.112859,0.135131,0.302451,-0.067289,0.030090,0.020095,0.107327,0.021946,0.430170,0.062794,0.427362,-0.139562,0.027472,0.131419,-0.050279,-0.326841,-0.318972,0.196475,0.185676,-0.155374,0.060176,-0.012004,-0.067358,0.002503,0.127649,0.152596,-0.037731,-0.267700,-0.274499,-0.006970,-0.184752,0.054932,0.015867,0.109912,-0.000082,...,0.121164,-0.217894,-0.048373,0.191506,-0.125046,-0.204126,0.187196,-0.144931,-0.183426,-0.031838,-0.176233,0.192707,0.253268,0.159180,-0.051619,0.110963,0.467664,0.136564,-0.116687,-0.194654,-0.225855,0.060305,-0.182051,-0.094858,-5.753145,-0.430544,-0.096196,-0.037099,0.009764,-0.019659,-0.022370,-0.321931,0.071652,-0.239595,-0.114711,0.239941,0.107733,0.061932,0.629441,0.417959
2,0.089606,-0.114452,0.157323,0.015657,0.152667,-0.352516,0.098448,0.380798,-0.114708,-0.244245,-0.086395,0.037757,-0.010880,0.320740,0.162972,0.101190,-0.046626,0.213162,0.219105,-0.027633,-0.113588,-0.099889,0.089314,0.099486,-0.135969,-0.216305,-0.175764,0.225866,0.068100,-0.020560,-0.018281,0.051542,-0.388599,-0.336958,0.086325,0.039984,0.068570,-0.054057,-0.093390,-0.070891,...,0.031014,-0.044603,-0.085249,-0.020924,0.009540,-0.146324,-0.087281,-0.330616,-0.140934,-0.185638,-0.060687,0.150497,0.058385,0.254196,-0.014072,0.113893,0.395107,0.282438,0.174987,-0.053787,-0.397886,0.098765,-0.180523,0.072121,-6.962331,-0.279294,-0.121245,0.065999,-0.156699,-0.217195,-0.229150,-0.178881,0.234193,-0.139085,0.037097,0.246068,-0.012880,-0.071799,0.529332,0.380615
3,-0.125375,-0.058545,-0.129429,-0.107628,-0.304105,-0.203190,0.226137,0.158711,-0.076678,-0.044398,-0.296696,-0.127843,-0.045425,0.259921,0.199458,0.340376,-0.091153,0.130662,0.087126,-0.264176,0.094333,0.073670,-0.106589,0.040658,-0.055048,0.025232,0.191902,-0.043672,0.064993,0.178928,0.262746,0.059840,-0.006262,-0.329158,0.232546,-0.277781,-0.035381,-0.086124,0.304522,0.010976,...,0.088285,0.213585,0.108151,-0.022180,-0.364827,-0.195048,-0.051387,-0.199094,-0.047412,-0.091227,-0.004511,0.192218,-0.063572,0.051961,0.028780,0.095687,0.621607,0.192603,-0.127793,0.048106,-0.290899,-0.117908,-0.098175,0.256368,-6.155948,-0.370273,-0.142302,-0.025362,-0.047146,-0.234720,0.103592,-0.125050,-0.055821,0.000998,0.037700,0.044798,-0.067623,0.081697,0.446625,0.445961
4,0.131456,0.003370,-0.001269,-0.048962,-0.183062,-0.152779,0.217920,0.235372,0.097770,-0.090909,-0.033670,-0.107399,-0.164156,0.628630,0.195057,0.367297,-0.146439,0.111491,0.172675,-0.236713,-0.058929,-0.183212,0.299020,0.155977,-0.165793,0.111152,0.065651,-0.026628,-0.079303,0.171861,0.125053,-0.121696,-0.276433,-0.254619,0.232073,-0.268342,-0.129371,0.057542,0.080998,-0.122877,...,0.036485,-0.124530,0.070041,0.013081,-0.248389,-0.204709,0.088085,-0.245095,0.023698,-0.166536,-0.107585,0.232298,0.110611,0.285307,0.102113,0.280949,0.367623,0.122324,0.011049,-0.145418,-0.296109,-0.057227,-0.062689,0.102168,-6.480539,-0.287889,-0.188958,0.200361,0.1310

In [ ]:
train_dataloader = pd.concat([train_features, train_labels], axis=1, join="inner")
test_dataloader = pd.concat([test_features, test_labels], axis=1, join="inner")

TypeError: ignored

In [ ]:
model = LogisticRegression()
model.fit(train_features, train_labels)

LogisticRegression()

In [ ]:
model.score(test_features, test_labels)

0.846

In [ ]:
sentiment = {0:"Negative", 1:"Positive"}

# Dataset Parameters
input_nodes = 768
classes = 2
batch_size = 32


In [ ]:
def accuracy(data_loader, model, is_test):
    correct = 0
    samples = 0
    cm = np.zeros((10, 10))
    model.eval()

    with torch.no_grad():
        for data_features, labels in data_loader:
            data_features = data_features.to(device=dev)
            labels = labels.to(device=dev)
            data_features = data_features.reshape(data_features.shape[0], -1)

            outputs = model(data_features)
            _, predictions = outputs.max(1)
            correct += (predictions == labels).sum()
            samples += predictions.size(0)
            acc = correct/samples
            if is_test:
              cm1 = confusion_matrix(predictions.cpu().numpy(),labels.cpu().numpy())
              if cm1.shape == (10, 10):
                cm += cm1

    model.train()
    return acc, cm

In [ ]:
# Train Network
def trainer(model, train_dataloader, test_dataloader, epochs, loss_fn, optimizer):
  
  losses = []
  accuracies = []
  test_acc = 0.0
  
  for epoch in range(epochs):
    total_loss = 0.0
    samples = 0
    for batch_id, (data, labels) in enumerate(tqdm(train_dataloader)):
        
        # Use cuda
        data = data.to(device=dev)
        labels = labels.to(device=dev)

        # Reshape to flatten
        data = data.reshape(data.shape[0], -1)

        # forward
        outputs = model(data)
        if isinstance(loss_fn, nn.NLLLoss):
          outputs = f.log_softmax(outputs,dim=1)
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()*outputs.size(0)
        samples += outputs.size(0)

        # calculate gradients of loss function
        optimizer.zero_grad()
        loss.backward()

        # adam step or gradient descent
        optimizer.step()
    
    acc, _ = accuracy(train_dataloader, model, False)
    avg_loss = total_loss/samples

    losses.append(avg_loss)
    accuracies.append(acc)

    print(f"Epoch: {epoch+1} | Accuracy on training set: {acc*100:.2f} | Loss: {avg_loss:.5f}")

  test_acc, cm = accuracy(test_dataloader, model, True)

  return losses, accuracies, test_acc, cm

In [ ]:
def draw_graphs(losses, accuracies, epochs, cm):
  x = np.array(range(1,epochs+1))

  losses = np.array(losses)

  fig = plt.figure()
  plt.plot(x, losses)
  fig.suptitle('Loss Vs. Epochs')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.xticks(x)
  plt.show()

  accuracies = np.array(accuracies)*100


  fig = plt.figure()
  plt.plot(x, accuracies)
  fig.suptitle('Accuracy Vs. Epochs')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy (in %)')
  plt.xticks(x)
  plt.show()

  cmd = ConfusionMatrixDisplay(cm, display_labels=[label for key, label in clothes.items()])
  fig, ax = plt.subplots(figsize=(10,10))
  cmd.plot(ax=ax)

In [ ]:
def metrics(cm):
  true_positive = cm.diagonal()
  false_positive = cm.sum(0) - true_positive
  false_negative = cm.sum(1) - true_positive

  precision = true_positive / (true_positive + false_positive)
  recall = true_positive / (true_positive + false_negative)

  f_score = 2*(precision*recall)/(precision+recall)
  return precision, recall, f_score

In [ ]:
test_accuracies = []

In [ ]:
# KL Divergence Model and its parameters
class NeuralNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNet, self).__init__()
        self.hidden_layer1 = nn.Linear(input_size, 16)
        self.output_layer = nn.Linear(16, num_classes)

    def forward(self, x):
        # Activation Functions
        x = f.relu(self.hidden_layer1(x))
        x = torch.sigmoid(self.output_layer(x))
        return x


# Initialize network
model = NeuralNet(input_size=input_nodes, num_classes=classes).to(dev)

# Hyperparameters
epochs = 2
learning_rate = 0.0001
loss_fn = nn.NLLLoss()   # Loss Function
# loss_fn = nn.CrossEntropyLoss()    # Loss Function

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Results
losses, accuracies, test_acc, cm = trainer(model, train_dataloader, test_dataloader, epochs, loss_fn, optimizer)
p, r, fs = metrics(cm)

print("-"*10 + "Test Results" + "-"*10)
print(f"Accuracy on test set: {test_acc*100:.2f}")
print(f"Precision: {p}")
print(f"Recall: {r}")
print(f"F1 score: {fs}")

test_accuracies.append(test_acc)

draw_graphs(losses, accuracies, epochs, cm)

  0%|          | 0/1500 [00:00<?, ?it/s]


ValueError: ignored